In [1]:
import sys
from pathlib import Path

# Go one level up from "src"
project_root = Path().resolve().parent
sys.path.append(str(project_root))

In [2]:
from src.database import DataBase

print(DataBase.__name__)

DataBase


In [3]:
from src.database import get_engine, DataBase
from sqlalchemy import String, BigInteger 

# Define columns for the "companies" table
columns = {
    "ticker": ((String,), {"unique": True, "nullable": False}),
    "longName": ((String,), {}),
    "sector": ((String,), {}),
    "industry": ((String,), {}),
    "country": ((String,), {}),
    "marketCap": ((BigInteger,), {}),
    "fullTimeEmployees": ((BigInteger,), {}),
    "website": ((String,), {}),
}

# Create the engine
engine = get_engine()

# Dynamically create the companies table
DataBase.create_table(engine, "companies", columns)


Creating dynamic table: companies


In [3]:
from sqlalchemy import inspect
from src.database import get_engine, DataBase
# Create the engine
engine = get_engine()

# Create an inspector
inspector = inspect(engine)

# List all tables
tables = inspector.get_table_names()
print("Tables in database:", tables)

Tables in database: ['companies', 'daily_prices', 'daily_prices_adjusted']


In [ ]:
from src.database import get_engine, get_session
from src.etl.companies import CompanyMetadata

# Step 1: Define FAANG tickers
faang = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]

# Step 2: Create DB engine and session
engine = get_engine()
session = get_session(engine)

# Step 3: Loop through each ticker and save it into the dynamic table
for ticker in faang:
    company = CompanyMetadata(ticker)
    company.save_to_table(session, "companies")

session.close()
print("FAANG company metadata successfully inserted into table.")


FAANG company metadata successfully inserted into dynamic table.


In [3]:
from src.database import get_engine, get_session, DataBase
from src.etl.daily_prices import DailyPrices
from sqlalchemy import String, Date, Float, BigInteger

# Define the table schema
columns = {
    "ticker": ((String,), {"nullable": False}),
    "date": ((Date,), {"nullable": False}),
    "open": ((Float,), {}),
    "high": ((Float,), {}),
    "low": ((Float,), {}),
    "close": ((Float,), {}),
    "volume": ((BigInteger,), {})
}

engine = get_engine()
session = get_session(engine)

# Create the table dynamically
DataBase.create_table(engine, "daily_prices", columns)

# Pull and insert price data for FAANG from 2021-01-01 to 2024-12-31
faang = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]

for ticker in faang:
    dp = DailyPrices(ticker, start="2021-01-01", end="2024-04-17")
    dp.save_to_table(session, "daily_prices")

session.close()

Creating dynamic table: daily_prices


In [4]:
from src.database import get_engine, get_session, DataBase
from src.etl.metrics import MetricsCalculator
from sqlalchemy import String, Date, Float, BigInteger
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
from src.database import metadata

# Define FAANG tickers
faang = ["META", "AAPL", "AMZN", "NFLX", "GOOGL"]
end = datetime.today()
start = end - timedelta(days=365 * 3)

# Connect to DB
engine = get_engine()
session = get_session(engine)

# Define new enriched table schema
columns_adjusted = {
    "ticker": ((String,), {"nullable": False}),
    "date": ((Date,), {"nullable": False}),
    "open": ((Float,), {}),
    "high": ((Float,), {}),
    "low": ((Float,), {}),
    "close": ((Float,), {}),
    "volume": ((BigInteger,), {}),
    "ma_5": ((Float,), {}),
    "ma_63": ((Float,), {}),
    "ma_126": ((Float,), {}),
    "ma_252": ((Float,), {}),
    "volatility_30d": ((Float,), {}),
    "macd": ((Float,), {}),
    "rsi": ((Float,), {})
}

# Create adjusted table
DataBase.create_table(engine, "daily_prices_adjusted", columns_adjusted)

# Pull, transform, and insert
for ticker in faang:
    df = yf.Ticker(ticker).history(start=start, end=end).reset_index()
    df["ticker"] = ticker
    df.rename(columns={
        "Date": "date",
        "Open": "open",
        "High": "high",
        "Low": "low",
        "Close": "close",
        "Volume": "volume"
    }, inplace=True)

    metrics = MetricsCalculator(df)
    enriched_df = metrics.get_adjusted_table()

    with engine.connect() as conn:
        records = enriched_df.to_dict(orient="records")
        conn.execute(metadata.tables["daily_prices_adjusted"].insert(), records)
        conn.commit()

session.close()
print("daily_prices_adjusted table successfully loaded.")


Creating dynamic table: daily_prices_adjusted
daily_prices_adjusted table successfully loaded.
